In [1]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
import numpy as np
import skquant.opt as skq

from qiskit.circuit import QuantumCircuit, ClassicalRegister, QuantumRegister, Qubit, Gate, Parameter, ParameterVector
from qiskit.tools.visualization import circuit_drawer 
from qiskit import Aer, execute
from qiskit.quantum_info import Operator, Pauli
from qiskit.opflow import PauliOp, SummedOp, ComposedOp
from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn, AerPauliExpectation, MatrixExpectation
from qiskit.algorithms import MinimumEigensolver, NumPyMinimumEigensolver, VQE
from qiskit.algorithms.optimizers import BOBYQA, COBYLA, GradientDescent, SLSQP, SciPyOptimizer
from qiskit.providers.aer import *

from scipy.optimize import minimize

import itertools

In [2]:
##Specify backend for simulation
backend = AerSimulator(method='statevector', device='GPU', cuStateVec_enable=True)

In [3]:
##Qubits on Lattice, Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Nq = len(qubits)

xx_links = [(1, 14), (2, 13), (12, 19), (5, 10), (6, 9), (8, 23), (11, 20), (15, 16), (21, 26), (22, 25), (17, 30), (18, 29)]
yy_links = [(1, 2), (3, 4), (5, 6), (10, 9), (12, 11), (14, 13), (17, 18), (19, 20), (21, 22), (26, 25), (28, 27), (30, 29)]
yy_links_reversed = [(30, 29), (28, 27), (26, 25), (21, 22), (19, 20), (17, 18), (14, 13), (12, 11), (10, 9), (5, 6), (3, 4), (1, 2)]
zz_links = [(0, 1), (2, 3), (4, 5), (6, 7), (9, 8), (11, 10), (13, 12), (15, 14), (16, 17), (18, 19), (20, 21), (22, 23), (25, 24), (27, 26), (29, 28), (31, 30)]
zz_links_reversed = [(31, 30), (29, 28), (27, 26), (25, 24), (22, 23), (20, 21), (18, 19), (16, 17), (15, 14), (13, 12), (11, 10), (9, 8), (6, 7), (4, 5), (2, 3), (0, 1)]

Jx, Jy, Jz = -1.0/np.sqrt(2), -1.0/np.sqrt(2), -1.0
hX = hY = hZ = 0.05/np.sqrt(3)

In [4]:
##Hamiltonian Construction (Square-Octagon Model)

#Construct Pauli strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = ""
    for q in range(Nq):
        if q != i and q != j:
            XX_string += 'I'
        else:
            XX_string += 'X'
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = ""
    for q in range(Nq):
        if q != i and q != j:
            YY_string += 'I'
        else:
            YY_string += 'Y'
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = ""
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string += 'I'
        else:
            ZZ_string += 'Z'
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = ""
    Y_string = ""
    Z_string = ""
    for q in range(Nq):
        if q != k:
            X_string += 'I'
            Y_string += 'I'
            Z_string += 'I'
        else:
            X_string += 'X'
            Y_string += 'Y'
            Z_string += 'Z'
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [5]:
##Hamiltonian Construction (Square-Octagon Model) (ctd.)

#Construct individual terms of Hamiltonian as Nq-qubit Pauli Operators
XX_links_terms = []
YY_links_terms = []
ZZ_links_terms = []
for i in XX_links_strings:
    XX_links_terms.append(PauliOp(Pauli(i), coeff=float(Jx)))
for i in YY_links_strings:
    YY_links_terms.append(PauliOp(Pauli(i), coeff=float(Jy)))
for i in ZZ_links_strings:
    ZZ_links_terms.append(PauliOp(Pauli(i), coeff=float(Jz)))
    
X_qubit_terms = []
Y_qubit_terms = []
Z_qubit_terms = []
for i in X_qubit_strings:
    X_qubit_terms.append(PauliOp(Pauli(i), coeff=float(hX)))
for i in Y_qubit_strings:
    Y_qubit_terms.append(PauliOp(Pauli(i), coeff=float(hY)))
for i in Z_qubit_strings:
    Z_qubit_terms.append(PauliOp(Pauli(i), coeff=float(hZ)))
    
#Construct Hamiltonian (for running with built-in VQE)
Hamiltonian = 0
for i in XX_links_terms:
    Hamiltonian += i
for i in YY_links_terms:
    Hamiltonian += i
for i in ZZ_links_terms:
    Hamiltonian += i
for i in X_qubit_terms:
    Hamiltonian += i
for i in Y_qubit_terms:
    Hamiltonian += i
for i in Z_qubit_terms:
    Hamiltonian += i

In [6]:
##Hamiltonian Variational Ansatz Circuit Construction
def HVA_circuit(n_layers):
    global xx_links
    global yy_links
    global zz_links
    global qubits
    #Declare variational parameters
    alpha_mag = ParameterVector(r'$\tilde{\alpha}$', n_layers)
    beta_mag = ParameterVector(r'$\tilde{\beta}$', n_layers)
    gamma_mag = ParameterVector(r'$\tilde{\gamma}$', n_layers)
    alpha = ParameterVector(r'$\alpha$', n_layers)
    beta = ParameterVector(r'$\beta$', n_layers)
    gamma = ParameterVector(r'$\gamma$', n_layers)
    #Initialize circuit
    HVA = QuantumCircuit(Nq)
    for idx_layer in range(n_layers):
    #Construct X-terms portion of layer
        for q in qubits:
            HVA.rx(2*alpha_mag[idx_layer], q)
        for (i,j) in xx_links:
            HVA.h(i)
            HVA.h(j)
            HVA.cx(max(i,j), min(i,j))
            HVA.rz(2*alpha[idx_layer], min(i,j))
            HVA.cx(max(i,j), min(i,j))
            HVA.h(i)
            HVA.h(j)
        HVA.barrier()
    #Construct Y-terms portion of layer
        for q in qubits:
            HVA.ry(2*beta_mag[idx_layer], q)
        for (i,j) in yy_links:
            HVA.rx(np.pi/2, i)
            HVA.rx(np.pi/2, j)
            HVA.cx(max(i,j), min(i,j))
        for (i,j) in yy_links_reversed:
            HVA.rz(2*beta[idx_layer], min(i,j))
            HVA.cx(max(i,j), min(i,j))
            HVA.rx(-np.pi/2,i)
            HVA.rx(-np.pi/2,j)
        HVA.barrier()
    #Construct Z-terms portion of layer
        for q in qubits:
            HVA.rz(2*gamma_mag[idx_layer], q)
        for (i,j) in zz_links:
            HVA.cx(max(i,j), min(i,j))
        for (i,j) in zz_links_reversed:
            HVA.rz(2*gamma[idx_layer], min(i,j))
            HVA.cx(max(i,j), min(i,j))
        HVA.barrier()
    return HVA

In [7]:
##Perform VQE (Using Built-in VQE Algorithm in Qiskit)

#Specify classical optimizer
optimizer = COBYLA(maxiter=50000)

def callback(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)
    
np.random.seed(10) #Seed for reproducibility
    
##Execution
initial_point = np.random.uniform(0.0, 2*np.pi, size=HVA_circuit(1).num_parameters)
local_vqe = VQE(ansatz=HVA_circuit(1),
                optimizer=optimizer,
                initial_point=initial_point,
                quantum_instance=backend)

local_result_1 = local_vqe.compute_minimum_eigenvalue(Hamiltonian)
local_result_2 = local_vqe.get_optimal_circuit()
print(local_result_1)
print(local_result_2)

Note: Using a statevector_simulator with 32 qubits can be very expensive. Consider using the Aer qasm_simulator instead to take advantage of Aer's built-in fast Pauli Expectation


ValueError: 'to_matrix' will return an exponentially large matrix, in this case '4294967296x4294967296' elements. Set algorithm_globals.massive=True or the method argument massive=True if you want to proceed.